In [1]:
import twd97
from tqdm.autonotebook import tqdm
import pandas as pd
import numpy as np
from pathlib import Path

C:\Users\Xunhaoz\AppData\Local\Temp\ipykernel_22368\1266993320.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
data_path = 'private_dataset.csv'

In [3]:
training_data = pd.read_csv(data_path)

lat = []
lng = []
for E, N in tqdm(zip(training_data['橫坐標'], training_data['縱坐標']), total=len(training_data)):
    latitude, longitude = twd97.towgs84(E, N)
    lat.append(latitude)
    lng.append(longitude)

training_data['lat'] = lat
training_data['lng'] = lng

print("房市訓練資料有：", len(training_data), "筆")
training_data

  0%|          | 0/5875 [00:00<?, ?it/s]

房市訓練資料有： 5875 筆


,ID,縣市,鄉鎮市區,路名,土地面積,使用分區,移轉層次,總樓層數,主要用途,主要建材,...,車位面積,車位個數,橫坐標,縱坐標,備註,主建物面積,陽台面積,附屬建物面積,lat,lng
0,PR-1,高雄市,前鎮區,鎮海路,-0.541033,NaN,12,14,住家用,鋼筋混凝土造,...,-0.819326,0.0,179750,2499123,NaN,0.127159,0.035249,-0.438452,22.590707,120.316770
1,PR-2,台北市,北投區,大同街,0.753799,NaN,4,7,其他,鋼筋混凝土造,...,0.311470,1.0,300436,2781335,NaN,-0.464083,-0.160979,-0.438452,25.139613,121.500226
2,PR-3,台北市,大同區,大龍街,0.482869,住,6,7,住家用,鋼筋混凝土造,...,-0.819326,0.0,302091,2773374,NaN,0.832407,2.461656,-0.438452,25.067685,121.516338
3,PR-4,新北市,蘆洲區,民族路,0.390940,NaN,4,4,住家用,加強磚造,...,-0.819326,0.0,298394,2775413,NaN,0.052476,-1.642077,-0.438452,25.086216,121.479765
4,PR-5,新北市,新莊區,龍安路,-0.711599,NaN,3,8,住家用,鋼筋混凝土造,...,-0.819326,0.0,292572,2767887,NaN,-0.968196,-0.333319,-0.022099,25.018446,121.421816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,PR-5871,高雄市,鳳山區,和德街,-0.163249,NaN,25,25,集合住宅,鋼筋混凝土造,...,1.661628,2.0,184250,2502080,NaN,-0.496050,0.946431,0.870392,22.617591,120.360410
5871,PR-5872,台北市,南港區,玉成街,1.118749,NaN,6,17,一般事務所,其他,...,4.845468,3.0,308685,2771134,NaN,2.717029,1.105120,-0.130991,25.047221,121.581601
5872,PR-5873,新北市,林口區,文化二路一段,-0.053200,NaN,14,14,住家用,鋼筋混凝土造,...,-0.819326,0.0,287075,2773585,NaN,0.242296,-0.084194,-0.438452,25.070033,121.367504
5873,PR-5874,高雄市,鳳山區,文龍東路,-0.089851,NaN,2,15,集合住宅,鋼筋混凝土造,...,-0.819326,0.0,185561,2504746,NaN,-0.257007,2.461656,-0.438452,22.641717,120.373053


In [4]:
def count_around_build(base_df: pd.DataFrame, query_df: pd.DataFrame, lat_name: str, lng_name: str,
                       threshold: float) -> list:
    base_lat_numpy = base_df['lat'].values
    base_lng_numpy = base_df['lng'].values
    query_lat_numpy = query_df[lat_name].values
    query_lng_numpy = query_df[lng_name].values

    def geo_distance(a, b):
        earth_radius = 6371.009

        a_lat, a_lng = a
        b_lat, b_lng = b

        d_lat = np.radians(b_lat - a_lat)
        d_lng = np.radians(b_lng - a_lng)

        h = (np.sin(d_lat / 2) ** 2 + np.cos(np.radians(a_lat)) * np.cos(np.radians(b_lat)) * (np.sin(d_lng / 2) ** 2))
        d = 2 * earth_radius * np.arctan2(np.sqrt(h), np.sqrt(np.ones_like(h) - h))
        return d

    static_list = []
    for lat, lng in tqdm(zip(base_lat_numpy, base_lng_numpy), total=len(base_df)):
        dist = geo_distance((lat, lng), (query_lat_numpy, query_lng_numpy))
        static_list.append((dist < threshold).sum())

    return static_list

# 全資料統計
## 更改KM即可

In [5]:
# KM = 1
#
# external_data = Path('external_data')
# for institute in external_data.glob('*.csv'):
#     name = institute.name.split('.')[0].replace('基本資料', '').replace('資料', '')
#     name = "鄰近" + name + "統計"
#     print(name)
#
#     df = pd.read_csv(institute)
#     print("資料長度：", len(df))
#     static_list = count_around_build(training_data, df, 'lat', 'lng', KM)
#     training_data[name] = static_list
#
# training_data.to_csv(f'{KM}KM_' + data_path)

# 自定統計

In [6]:
institute = Path('external_data/捷運站點資料.csv')
name = institute.name.split('.')[0].replace('基本資料', '').replace('資料', '')
name = "鄰近" + name + "統計"
print(name)

df = pd.read_csv(institute)
print("資料長度：", len(df))
static_list = count_around_build(training_data, df, 'lat', 'lng', 0.5)
training_data[name] = static_list

鄰近捷運站點統計
資料長度： 239


  0%|          | 0/5875 [00:00<?, ?it/s]

In [7]:
institute = Path('external_data/火車站點資料.csv')
name = institute.name.split('.')[0].replace('基本資料', '').replace('資料', '')
name = "鄰近" + name + "統計"
print(name)

df = pd.read_csv(institute)
print("資料長度：", len(df))
static_list = count_around_build(training_data, df, 'lat', 'lng', 1)
training_data[name] = static_list

鄰近火車站點統計
資料長度： 242


  0%|          | 0/5875 [00:00<?, ?it/s]

In [8]:
institute = Path('external_data/公車站點資料.csv')
name = institute.name.split('.')[0].replace('基本資料', '').replace('資料', '')
name = "鄰近" + name + "統計"
print(name)

df = pd.read_csv(institute)
print("資料長度：", len(df))
static_list = count_around_build(training_data, df, 'lat', 'lng', 0.2)
training_data[name] = static_list

鄰近公車站點統計
資料長度： 111761


  0%|          | 0/5875 [00:00<?, ?it/s]

In [9]:
institute = Path('external_data/腳踏車站點資料.csv')
name = institute.name.split('.')[0].replace('基本資料', '').replace('資料', '')
name = "鄰近" + name + "統計"
print(name)

df = pd.read_csv(institute)
print("資料長度：", len(df))
static_list = count_around_build(training_data, df, 'lat', 'lng', 0.2)
training_data[name] = static_list

鄰近腳踏車站點統計
資料長度： 6276


  0%|          | 0/5875 [00:00<?, ?it/s]

In [10]:
institute = Path('external_data/國小基本資料.csv')
name = institute.name.split('.')[0].replace('基本資料', '').replace('資料', '')
name = "鄰近" + name + "統計"
print(name)

df = pd.read_csv(institute)
print("資料長度：", len(df))
static_list = count_around_build(training_data, df, 'lat', 'lng', 1)
training_data[name] = static_list

鄰近國小統計
資料長度： 2668


  0%|          | 0/5875 [00:00<?, ?it/s]

In [11]:
institute = Path('external_data/國中基本資料.csv')
name = institute.name.split('.')[0].replace('基本資料', '').replace('資料', '')
name = "鄰近" + name + "統計"
print(name)

df = pd.read_csv(institute)
print("資料長度：", len(df))
static_list = count_around_build(training_data, df, 'lat', 'lng', 1)
training_data[name] = static_list

鄰近國中統計
資料長度： 965


  0%|          | 0/5875 [00:00<?, ?it/s]

In [12]:
institute = Path('external_data/醫療機構基本資料.csv')
name = institute.name.split('.')[0].replace('基本資料', '').replace('資料', '')
name = "鄰近" + name + "統計"
print(name)

df = pd.read_csv(institute)
print("資料長度：", len(df))
static_list = count_around_build(training_data, df, 'lat', 'lng', 0.5)
training_data[name] = static_list

鄰近醫療機構統計
資料長度： 23302


  0%|          | 0/5875 [00:00<?, ?it/s]

In [13]:
institute = Path('external_data/便利商店.csv')
name = institute.name.split('.')[0].replace('基本資料', '').replace('資料', '')
name = "鄰近" + name + "統計"
print(name)

df = pd.read_csv(institute)
print("資料長度：", len(df))
static_list = count_around_build(training_data, df, 'lat', 'lng', 0.5)
training_data[name] = static_list

鄰近便利商店統計
資料長度： 13097


  0%|          | 0/5875 [00:00<?, ?it/s]

In [14]:
training_data.to_csv(f'Custom_' + data_path)